# intro

[intro bit]

# preliminary tool research


[intro to our assigned pipeline tools]

## minimap2
### 8-3-25

Minimap2 is a readmapping tool that is specifically suited for our project due to it's compaibility with the sequencing data. This is due to the fact that most of our earlier experience was with illumina based sequencing whilst this project handles MinION data.

pros:

- Minimap2 is especialy suited for readmapping MinION data because of it's explicit design for the high error rate of MinION reads. (This is due to its method of mapping reads using k-mers) K-mers are essentially genetic reading frames where the k represents the size of the frame in nucleotides.

- Minimap2 is very efficient in the way that it has a comparatively low memory usage compared to other high performance tools meant for readmapping. (This is due to the fact that it was written in C, which is a very computationally efficient programming language)

- Minimap2 supports spliced alignment. Which means it can detect large gaps caused by structural variations, introns etc.

- Minimap2 is very suited for refferencing reads against large databases such as RefSeq. This is because it uses approximated alignment to not require single nucleotide level accuracy for every alignment.

cons:

- Minimap2 is primarily designed for readmapping for alignment instead of taxonomic classification like some of the other tools (which will be named below) that have comparable functions and goals.

- Minimap2 is not very suited for unexpected or divergent microbial genomes.

- Minimap2 needs more post processing correction than other tools due to approximated alignment


#### k-mer
K-mers are essentially breaking down reads into smaller more easily findable pieces. 

example:
```
Imagine you have a read of 5 nucleotides in length such as "ATGCT".

Now if you assign the k in k-mer to be 3 then the k-mers would essentially be the sequences "ATG", "TGC" and "GCT".
```

#### approximated alignment
The minimap2 tool is more efficient because it uses these smaller and more computationally efficient k-mer pieces to match reads to the references.
The way it matches reads to the reference genomes is by using multiple k-mer matches, so it won't have to compare every single base.

Note that not all the possible k-mers are used so it's not exponentially more pieces than conventional single base granularity readmapping.

After enough k-mers are matched to the reference genome a more granular base level alignment will take place to confirm.

Note that this does result in more false positives, due to which this tool requires more post processing than other tools with the same purpose.



## readmapping alternatives
[mention al in one sollution preferences]


aio solutions (readmapping & taxonomic sorting):
minimap
	needs custom dataset for referencing and aligning, but most efficient and accurate in readmapping
    
kraken2
	prebuilt dataset and recced for general taxonomy 

mmseq2
	https://github.com/soedinglab/MMseqs2 very efficient blast alternative

metaphlan
	https://github.com/biobakery/MetaPhlAn microbial (strain level so very detailed, same species variants)




# kraken2

### 15-03-25

alternative to 

pros:
- built in taxonomic support
- extensive documentation
- generally used in the papers we reference

cons:
- not as accurate and efficient as minimap2

final verdict:
better alternative because minimap2 is overqualified for our project and timeframe

[github](https://github.com/DerrickWood/kraken2)

## bracken

extension upon kraken for 16s data, refines classification using bayesian re estimation to better species level classification. summary, makes more accurate kraken.

[github](https://github.com/jenniferlu717/Bracken)

<blockquote>
    <p>Bracken (Bayesian Reestimation of Abundance with KrakEN) is a highly accurate statistical method that computes the abundance of species in DNA sequences from a metagenomics sample. Braken uses the taxonomy labels assigned by Kraken, a highly accurate metagenomics classification algorithm, to estimate the number of reads originating from each species present in a sample. Kraken classifies reads to the best matching location in the taxonomic tree, but does not estimate abundances of species. We use the Kraken database itself to derive probabilities that describe how much sequence from each genome is identical to other genomes in the database, and combine this information with the assignments for a particular sample to estimate abundance at the species level, the genus level, or above. Combined with the Kraken classifier, Bracken produces accurate species- and genus-level abundance estimates even when a sample contains two or more near-identical species.

        NOTE: Bracken is compatible with both Kraken 1 and Kraken 2. However, the default kmer length is different depending on the version of Kraken used. If you use Kraken 1 defaults, specify 31 as the kmer length. If you use Kraken 2 defaults, specify 35 as the kmer length. </p>
</blockquote>


[documentation & source of quote above](https://ccb.jhu.edu/software/bracken/)

---
# readmapping and taxonomic classification (kraken2 & bracken)

## installing kraken2

In [ ]:
 conda install bioconda::kraken2

db path /data/datasets/KRAKEN2_INDEX
data path /commons/Themas/Thema07/metagenomics/bioplastic

installing bracken

In [ ]:
 conda install bioconda::kraken2

reference kraken en bracken commands in 16S_Silva README (copy in kraken2_test_data)

## acquire data

making a subset for easier testing

In [ ]:
cp /commons/Themas/Thema07/metagenomics/bioplastic/soil_metagenomics/Ariana/20240418_1521_MN35459_FAX77312_60832f1a/fastq_pass/barcode01/FAX77312_pass_barcode01_60832f1a_527e8671_0.fastq.gz ~/Documents/git/metagenomics/kraken2_test_data

gunzip file

In [ ]:
gunzip FAX77312_pass_barcode01_60832f1a_527e8671_0.fastq.gz

In [ ]:
cd ~/Documents/git/metagenomics/kraken2_test_data

make subset of 800.000 lines (lines must be multipe of 4 because of fastq syntax)

In [ ]:
head -n 800000 FAX77312_pass_barcode01_60832f1a_527e8671_0.fastq > test_data.fastq

## test kraken commands

In [ ]:
kraken2 --db 16S_Silva/ --threads 4 report ~/Documents/git/metagenomics/kraken2_test_data/subset_test.kreport2 ~/Documents/git/metagenomics/kraken2_test_data/test_data.fastq

above command badly formatted report option

In [ ]:
kraken2 --db 16S_Silva/ --threads 4   --report ~/Documents/git/metagenomics/kraken2_test_data/subset_test.kreport2   --output ~/Documents/git/metagenomics/kraken2_test_data/output.txt   ~/Documents/git/metagenomics/kraken2_test_data/test_data.fastq

kraken2: database ("16S_Silva/") does not contain necessary file taxo.k2d, but still generated output files

## snakemake adaptation
given that the preliminary processing of wgs barcode01 has succeeded as stated in the section preliminary kraken2 processing, I will now start with the implementation of the command into a snakemake rule

in the code below I have directly translated the latest version of the preliminary processing command into snakemake

In [ ]:
rule kraken2:
    input:
        "~/Documents/git/metagenomics/kraken2_test_data/combined_wgs_barcode01.fastq"
    output:
        report="combined_wgs_barcode01_report.kreport2",
        output_txt="combined_wgs_barcode01_output.txt"
    threads: 16
    shell:
        "kraken2 --db /data/datasets/KRAKEN2_INDEX/k2_standard "
        "--threads {threads} "
        "--report {output.report} "
        "--output {output.output_txt} "
        "{input}"


turns out that kraken2 recommends using relative paths from the snakefile so the next iteration below wil have this 

In [ ]:
rule all:
    input:
        "combined_wgs_barcode01_report.kreport2",
        "combined_wgs_barcode01_output.txt"



rule kraken2:
    input:
        "/kraken2_test_data/combined_wgs_barcode01.fastq"
    output:
        kreport="combined_wgs_barcode01_report.kreport2",
        output_txt="combined_wgs_barcode01_output.txt"
    threads: 16
    shell:
        "kraken2 --db /data/datasets/KRAKEN2_INDEX/k2_standard "
        "--threads {threads} "
        "--report {output.kreport} "
        "--output {output.output_txt} "
        "{input}"

the code above returns a MissingInputException specifically for 
```host: mordor2014
Building DAG of jobs...
MissingInputException in rule kraken2 in file "/homes/ttvo/Documents/git/metagenomics/snakefile", line 8:
Missing input files for rule kraken2:
    output: combined_wgs_barcode01_report.kreport2, combined_wgs_barcode01_output.txt
    affected files:
        /kraken2_test_data/combined_wgs_barcode01.fastq```

this is because I moved the ```combined_wgs_barcode01.fastq``` file to an accessible place.
I retrieved a copy of the data with the following command

In [ ]:
cp /students/2024-2025/Thema07/metagenomics/bioplastic/combined_wgs_barcode1.fastq ~/Documents/git/metagenomics/kraken2_test_data/

after trying to run the snakefile code above it became clear that I misspelled the file which is ```combined_wgs_barcode1.fastq``` as ```combined_wgs_barcode01.fastq```. 

After this the running of the file still resulted in a missing file and after another look at the input filepath of ```/kraken2_test_data/combined_wgs_barcode01.fastq``` I realised the filepath began with a ```/``` wihch resulted int the path not being relative. After this being corrected by removing the initial ```/```, the reulting output was satisfactory for a dry run. The code for the dry run and the resulting output are displayed below.

In [ ]:
(metagen) ttvo@assemblix2019:~/Documents/git/metagenomics$ snakemake -np
host: assemblix2019
Building DAG of jobs...
Job stats:
job        count
-------  -------
all            1
kraken2        1
total          2


[Mon Mar 24 04:44:50 2025]
rule kraken2:
    input: kraken2_test_data/combined_wgs_barcode1.fastq
    output: combined_wgs_barcode01_report.kreport2, combined_wgs_barcode01_output.txt
    jobid: 1
    reason: Missing output files: combined_wgs_barcode01_report.kreport2, combined_wgs_barcode01_output.txt
    threads: 16
    resources: tmpdir=<TBD>
Shell command: kraken2 --db /data/datasets/KRAKEN2_INDEX/k2_standard --threads 16 --report combined_wgs_barcode01_report.kreport2 --output combined_wgs_barcode01_output.txt kraken2_test_data/combined_wgs_barcode1.fastq

[Mon Mar 24 04:44:50 2025]
rule all:
    input: combined_wgs_barcode01_report.kreport2, combined_wgs_barcode01_output.txt
    jobid: 0
    reason: Input files updated by another job: combined_wgs_barcode01_output.txt, combined_wgs_barcode01_report.kreport2
    resources: tmpdir=<TBD>
Shell command: None
Job stats:
job        count
-------  -------
all            1
kraken2        1
total          2

Reasons:
    (check individual jobs above for details)
    input files updated by another job:
        all
    output files have to be generated:
        kraken2
This was a dry-run (flag -n). The order of jobs does not reflect the order of execution.

afterwards with log

In [ ]:
    log: "logs/kraken2/kreaken_log.txt"
    shell:
        "kraken2 --db /data/datasets/KRAKEN2_INDEX/k2_standard"
        "--threads {threads} "
        "--report {output.kreport} "
        "--output {output.output_txt} "
        "{input} 2> {log}"

this resulted in a lack of 

## preliminary kraken2 processing
### 20-04-25

I'm processing the wgs of barcode01 in advance for better efficiency during developement, specifically so that stijn can continue working with kraken2 output data.



navigate and merge all barcode01 fastafiles (using zcat to decompress before concatonating)

```cd /commons/Themas/Thema07/metagenomics/bioplastic/soil_metagenomics/Ariana/20240418_1521_MN35459_FAX77312_60832f1a/fastq_pass/barcode01```

```zcat FAX77312_pass_barcode01_60832f1a_527e8671_*.fastq.gz > ~/Documents/git/metagenomics/kraken2_test_data/combined_wgs_barcode1.fastq```

navigate towards new file 

```cd ~/Documents/git/metagenomics/kraken2_test_data/```

execute kraken on combined data

```kraken2 --db /data/datasets/KRAKEN2_INDEX/k2_standard --threads 16 --report combined_wgs_barcode01_report.kreport2 --output combined_wgs_barcode01_output.txt combined_wgs_barcode1.fastq```

moved combined fastq file to shared folder for easier access for collaborators with the commands below

```cd /students/2024-2025/Thema07/metagenomics/bioplastic```

```mv ~/Documents/git/metagenomics/kraken2_test_data/combined_wgs_barcode1.fastq .```


#### conclusion

Combined_wgs_barcode01_report.kreport2 and combined_wgs_barcode01_output.txt are present within the logbook folder of the git repository.

And the combined_wgs_barcode1.fastq is present within the /students/2024-2025/Thema07/metagenomics/bioplastic folder.

The goal of this section was providing adequate data for the further developement and integration of the bracken section. As of right now I deem it unneccesary to process more data for this purpose. However in the circumstance more adta is needed during developement, the above code should serve to reproduce the results on more barcodes
